# Выбор локации для скважины

Вы работаете в нефтедобывающей компании. Нужно решить, где бурить новую скважину.  

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль.
Шаги для выбора локации:
  
Основными этапами нашего проекта станут:

* Загрузка и подготовка данных
* Обучение и проверка модели
* Подготовка к расчёту прибыли
* Расчёт прибыли и рисков. Выбор региона для разработки месторождений

**Описание данных**

*Признаки*  
  
*id* — уникальный идентификатор скважины  
*f0, f1, f2* — три признака точек (нам неизвестно, что они означают, но сами признаки значимы)  
  
*Целевой признак*  
  
**product** — объём запасов в скважине (тыс. баррелей)  
  
Условия задачи:
Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).  
При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.  
Бюджет на разработку скважин в регионе — 10 млрд рублей.  
При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.  
После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.
  
Проект выполнен в **Jupyter Notebook**, версия сервера блокнотов: 6.1.4. Версия **Python** 3.7.8.
В проекте использованы библиотеки:
* **Pandas**
* **NumPy**
* **SciPy**
* **scikit-learn**
* **IPython**

## Загрузка и подготовка данных

In [1]:
# Импортируем все библиотеки и модули необходимые для выполнения проекта.
import pandas as pd
import numpy as np
from IPython.display import display
from scipy.stats import t
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Прочитаем датасеты.
data_0 = pd.read_csv('/datasets/geo_data_0.csv')
data_1 = pd.read_csv('/datasets/geo_data_1.csv')
data_2 = pd.read_csv('/datasets/geo_data_2.csv')
# Изучим датасеты.
for table in [data_0, data_1, data_2]:
    display(table)
    print(table['id'].value_counts())
    table.info()
    display(table.describe())
    print()
    print('Количество полных дубликатов равно', table.duplicated().sum())

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
...,...,...,...,...,...
99995,DLsed,0.971957,0.370953,6.075346,110.744026
99996,QKivN,1.392429,-0.382606,1.273912,122.346843
99997,3rnvd,1.029585,0.018787,-1.348308,64.375443
99998,7kl59,0.998163,-0.528582,1.583869,74.040764


bxg6G    2
AGS9W    2
TtcGQ    2
bsk9y    2
HZww2    2
        ..
0fQTe    1
ZprHc    1
QH0pf    1
525ui    1
3M54W    1
Name: id, Length: 99990, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347



Количество полных дубликатов равно 0


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
...,...,...,...,...,...
99995,QywKC,9.535637,-6.878139,1.998296,53.906522
99996,ptvty,-10.160631,-12.558096,5.005581,137.945408
99997,09gWa,-7.378891,-3.084104,4.998651,137.945408
99998,rqwUm,0.665714,-6.152593,1.000146,30.132364


...


Количество полных дубликатов равно 0


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
...,...,...,...,...,...
99995,4GxBu,-1.777037,1.125220,6.263374,172.327046
99996,YKFjq,-1.261523,-0.894828,2.524545,138.748846
99997,tKPY3,-1.199934,-2.957637,5.219411,157.080080
99998,nmxp2,-2.419896,2.417221,-5.548444,51.795253


xCHr8    2
VF7Jo    2
KUPhW    2
Vcm5J    2
cwcvm    1
        ..
bOrSf    1
vNRep    1
ItSSc    1
l99oH    1
7Rnvm    1
Name: id, Length: 99996, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838



Количество полных дубликатов равно 0


In [2]:
# Подготовим целевой и остальные признаки для каждого датасета.
# Напишем функцию split, которая разделит признаки датасетов.
def split(data):
    features = data.drop(['id','product'], axis=1)
    target = data['product']
    return features, target

features_0, target_0 = split(data_0)
features_1, target_1 = split(data_1)
features_2, target_2 = split(data_2)

### Вывод

Нами была проведена работа по загрузке, изучению и подготовке данных.  
  
  **Загрузка и изучение:**  
  В таблицах пропусков нет. Данные имеют корректные форматы. В столбце *id* имеются дубликаты, но их наличие оправдано.  
  
**Подготовка:**  
Определен целевой признак. Это объем запасов в скважине — столбец *product*. Для каждого датасета подготовлены наборы целевого и остальных признаков.

## Обучение и проверка модели

In [3]:
# Напишем функцию, которая будет делить наши выборки на тренировочный 
# и валидационный наборы, а также отображать их размеры.
def test_split_shape(features, target):
    features_train, features_valid, target_train, target_valid = train_test_split(
    features, 
    target, 
    test_size=0.25, 
    random_state=12345
    )
    print('Размер изначальной выборки features составляет', 
          features.shape)
    print('Размер тренировочной выборки features составляет', 
          features_train.shape)
    print('Размер валидационной выборки features составляет', 
          features_valid.shape)
    print('Размер изначальной выборки target составляет', 
          target.shape)
    print('Размер тренировочной выборки target составляет', 
          target_train.shape)
    print('Размер валидационной выборки target составляет', 
          target_valid.shape)
    return features_train, features_valid, target_train, target_valid

In [4]:
# Разобьем полученные датасеты на тренировочную и валидационную выборки
# с помощью нашей функции.
# Начнем с первого региона.
features_train_0, features_valid_0, target_train_0, target_valid_0 = test_split_shape(
    features_0, target_0
)

Размер изначальной выборки features составляет (100000, 3)
Размер тренировочной выборки features составляет (75000, 3)
Размер валидационной выборки features составляет (25000, 3)
Размер изначальной выборки target составляет (100000,)
Размер тренировочной выборки target составляет (75000,)
Размер валидационной выборки target составляет (25000,)


In [5]:
# Теперь второй регион.
features_train_1, features_valid_1, target_train_1, target_valid_1 = test_split_shape(
    features_1, target_1
)

Размер изначальной выборки features составляет (100000, 3)
Размер тренировочной выборки features составляет (75000, 3)
Размер валидационной выборки features составляет (25000, 3)
Размер изначальной выборки target составляет (100000,)
Размер тренировочной выборки target составляет (75000,)
Размер валидационной выборки target составляет (25000,)


In [6]:
# Теперь третий регион.
features_train_2, features_valid_2, target_train_2, target_valid_2 = test_split_shape(
    features_2, target_2
)

Размер изначальной выборки features составляет (100000, 3)
Размер тренировочной выборки features составляет (75000, 3)
Размер валидационной выборки features составляет (25000, 3)
Размер изначальной выборки target составляет (100000,)
Размер тренировочной выборки target составляет (75000,)
Размер валидационной выборки target составляет (25000,)


In [7]:
# Напишем функцию, которая будет создавать математическую модель
# линейной регрессии, обучать её и делать предсказания.
# Также включим в функцию отображение основных метрик.
def ml(feat_train, target_train, feat_valid, target_valid):
    model = LinearRegression()
    model.fit(feat_train, target_train)
    predictions = pd.Series(
    model.predict(feat_valid), 
    index=target_valid.index
    )
    # Посчитаем метрики MSE, RMSE, R2, MAE, 
    # а также средний запас предсказанного сырья.
    mse = mean_squared_error(target_valid, predictions)
    print('MSE нашей модели =', mse)
    print('RMSE нашей модели =', mse ** 0.5)
    print('R2 =', r2_score(target_valid, predictions))
    print('MAE =', mean_absolute_error(target_valid, predictions))
    print('Средний запас предсказанного сырья составил', 
          predictions.mean(), 'тыс. баррелей')
    return predictions

In [8]:
# Создадим модель, сделаем предсказания и посчитаем метрики.
# Начнем с первого региона.
# Для будущих расчетов сохраним предсказания в отдельную переменную.
predictions_0 = ml(
    features_train_0, 
    target_train_0, 
    features_valid_0, 
    target_valid_0
)

MSE нашей модели = 1412.2129364399243
RMSE нашей модели = 37.5794217150813
R2 = 0.27994321524487786
MAE = 30.919600777151313
Средний запас предсказанного сырья составил 92.59256778438035 тыс. баррелей


In [9]:
# Теперь второй регион.
# Для будущих расчетов сохраним предсказания в отдельную переменную.
predictions_1 = ml(
    features_train_1, 
    target_train_1, 
    features_valid_1, 
    target_valid_1
)

MSE нашей модели = 0.7976263360391157
RMSE нашей модели = 0.893099286775617
R2 = 0.9996233978805127
MAE = 0.7187662442124758
Средний запас предсказанного сырья составил 68.728546895446 тыс. баррелей


In [10]:
# И теперь третий регион.
# Для будущих расчетов сохраним предсказания в отдельную переменную.
predictions_2 = ml(
    features_train_2, 
    target_train_2, 
    features_valid_2, 
    target_valid_2
)

MSE нашей модели = 1602.3775813236196
RMSE нашей модели = 40.02970873393434
R2 = 0.20524758386040443
MAE = 32.792652105481814
Средний запас предсказанного сырья составил 94.96504596800489 тыс. баррелей


### Вывод

Мы разделили выборки `features` и `target` каждого региона на тренировочную и валидационную в соотношении 3:1. Выборкам были присвоены следующие переменные:  
1. *Первый регион:*  
   `features_train_0`  
   `target_train_0`  
   `features_valid_0`  
   `target_valid_0` 
2. *Второй регион:*  
   `features_train_1`  
   `target_train_1`  
   `features_valid_1`  
   `target_valid_1`
3. *Третий регион:*  
   `features_train_2`  
   `target_train_2`  
   `features_valid_2`  
   `target_valid_2` 

Мы создали модель линейной регрессии к каждому региону, и обучили её.  
Предсказания и правильные ответы для первого региона сохранены в переменных `predictions_0` и `target_valid_0` соответственно. Для второго и третьего регионов созданы переменные с аналогичным имененем, только вместо 0 указаны 1 и 2 соответственно.

Наивысшее качество показала модель во втором регионе. Метрика RMSE для неё составила 0,893 тыс. баррелей, а метрика R2 — 0,999.  
Модель можно назвать практически идеальной.  
**Самый большой средний предсказанный запас зафиксирован в третьем регионе. Он составил 94,965 тыс. баррелей. Правда стоит отметить, что качество этой модели худшее из трёх. Продолжим наши расчеты.**

## Подготовка к расчёту прибыли

In [11]:
# Создадим для каждого региона константы, 
# содержащие в себе константы для предсказаний.
PREDICTED_0 = predictions_0
PREDICTED_1 = predictions_1
PREDICTED_2 = predictions_2
# Также создадим константы для целевых показателей.
TARGET_0 = target_valid_0
TARGET_1 = target_valid_1
TARGET_2 = target_valid_2

Создадим константы для расчета достаточного объема запасов скважины для безубыточной разработки: 
* EXPLORED_WELLS — количество исследуемых точек в регионе
* BEST_WELLS — количество лучших отобранных точек в регионе
* WELL_DEVELOP_BUDGET — бюджет на разработку скважин в регионе. Указан в тыс. млн
* BARREL_INCOME — доход с каждой единицы продукта. Указан в тыс. рублей

In [12]:
EXPLORED_WELLS = 500
BEST_WELLS = 200
WELL_DEVELOP_BUDGET = 10_000_000
BARREL_INCOME = 450

In [13]:
# Посчитаем минимальное среднее количество продукта в месторождениях региона, 
# достаточное для безубыточной разработки скважины.
MIN_BREAK_EVEN_RESERVE = WELL_DEVELOP_BUDGET / BEST_WELLS / BARREL_INCOME
print('Минимальное количество продукта в месторождениях,'
    'достаточное для безубыточной разработки скважины составляет', 
      round(MIN_BREAK_EVEN_RESERVE), 'тыс. баррелей')
numbers = [
    PREDICTED_0.mean(), 
    PREDICTED_1.mean(), 
    PREDICTED_2.mean()
]
average_reserves = pd.DataFrame(
    data = numbers, 
    columns = ['Средний предсказанный запас'], 
    index = [1,2,3]
)
average_reserves.index.name = 'Регион'
display(average_reserves)

Минимальное количество продукта в месторождениях,достаточное для безубыточной разработки скважины составляет 111 тыс. баррелей


,Средний предсказанный запас
Регион,
1,92.592568
2,68.728547
3,94.965046


### Вывод

Мы рассчитали минимальное количество продукта в месторождениях, достаточное для безубыточной разработки скважины.  
Оно составило **111 тыс. баррелей.**  
Ближе всего к этому показателю близок 3 регион. Его средний предсказанный запас составил 95 тыс. баррелей.  
Средний предсказанный запас первого региона составил 93 тыс. баррелей.  
Средний предсказанный запас второго региона составил 69 тыс. баррелей.  

## Расчёт прибыли и рисков 

In [14]:
# Напишем функцию для расчета прибыли.
def profit(predictions, target, count):
    sorted_predictions = predictions.sort_values(ascending=False)
    sorted_target = target[sorted_predictions.index][:count]
    sorted_target_total = sorted_target.sum()
    result = sorted_target_total * BARREL_INCOME - WELL_DEVELOP_BUDGET
    return result

# Проверим работу функции и посчитаем прибыль по целевым параметрам 
# 200 лучших точек в каждом регионе.
print('Прибыль 200 лучших точек по целевому признаку в первом регионе составляет', 
      round(profit(PREDICTED_0, TARGET_0, 200)), 'тыс. руб')
print('Прибыль 200 лучших точек по целевому признаку во втором регионе составляет', 
      round(profit(PREDICTED_1, TARGET_1, 200)), 'тыс. руб')
print('Прибыль 200 лучших точек по целевому признаку в третьем регионе составляет', 
      round(profit(PREDICTED_2, TARGET_2, 200)), 'тыс. руб')

Прибыль 200 лучших точек по целевому признаку в первом регионе составляет 3320826 тыс. руб
Прибыль 200 лучших точек по целевому признаку во втором регионе составляет 2415087 тыс. руб
Прибыль 200 лучших точек по целевому признаку в третьем регионе составляет 2710350 тыс. руб


In [15]:
# Напишем функцию для применения техники bootstrap.
def bootstrap(predictions, target):
    state = np.random.RandomState(12345)
    values = []
    # При разведке региона используют 500 точек. n=500.
    for i in range(1000):
        predict_subsample = predictions.sample(
        n=500, 
        replace=True, 
        random_state=state
        )
        target_subsample = target[predict_subsample.index]
        values.append(profit(predict_subsample, target_subsample, 200))
    values = pd.Series(values)
    # Найдем среднюю прибыль.
    mean_value = values.mean()
    print('Средняя прибыль в регионе составляет', 
          round(mean_value), 'тыс. руб.')
    # Найдем 95 %-доверительный интервал.
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    print('95%-ый доверительный интервал:', 'от', 
          round(lower), 'до', round(upper))
    # Посчитаем как часто в этом регионе встречаются убыточные точки.
    risk = stats.percentileofscore(values, 0)
    print('Вероятность убытков в данном регионе составляет', risk, '%')

In [16]:
# Посчитаем распределение прибыли методом bootstrap, среднюю прибыль, 
# 95% доверительный интервал, а также риски убытков.
# Начнем с первого региона.
bootstrap(PREDICTED_0, TARGET_0)

Средняя прибыль в регионе составляет 425939 тыс. руб.
95%-ый доверительный интервал: от -102090 до 947976
Вероятность убытков в данном регионе составляет 6.0 %


In [17]:
# Теперь второй регион.
bootstrap(PREDICTED_1, TARGET_1)

Средняя прибыль в регионе составляет 515223 тыс. руб.
95%-ый доверительный интервал: от 68873 до 931548
Вероятность убытков в данном регионе составляет 1.0 %


In [18]:
# Наконец третий регион.
bootstrap(PREDICTED_2, TARGET_2)

Средняя прибыль в регионе составляет 435008 тыс. руб.
95%-ый доверительный интервал: от -128881 до 969707
Вероятность убытков в данном регионе составляет 6.4 %


### Вывод

Нами была подготовлена функция расчета прибыли `profit`. Данная функция принимает на вход предсказанный запас месторождения, правильные ответы и требуемое количество месторождений с максимальными запасами.  
Данная функция была применена во время расчета средней прибыли методом bootstrap.  
Для неё была создана функция `bootstrap`. Функция принимает на вход предсказания и правильные ответы. На выходе функция считает распределение прибыли методом bootstrap, считает среднюю прибыль, 95% доверительный интервал, а также риски убытков.  
  
  Результаты:  
* **Регион № 1**  
Средняя прибыль в первом регионе составляет 425939 тыс. руб.  
95%-ый доверительный интервал: от -102090 до 947976  
Вероятность убытков в данном регионе составляет 6.0 %
* **Регион № 2**  
Средняя прибыль в первом регионе составляет 515223 тыс. руб.  
95%-ый доверительный интервал: от 68873 до 931548 
Вероятность убытков в данном регионе составляет 1.0 %
* **Регион № 3**  
Средняя прибыль в первом регионе составляет 435008 тыс. руб.  
95%-ый доверительный интервал: от -128881 до 969707  
Вероятность убытков в данном регионе составляет 6.4 %  
  
Одним из условий проекта является фильтр регионов по вероятности убытков. Нам подойдут только регионы, у которых вероятность убытков менее 2,5 %. Таким регионом является регион № 2. Помимо этого данный регион обладает наибольшей средней прибылью — 515223 тыс. руб.  
**Вывод: по результатам проекта мы предлагаем для разработки скважи регион № 2.**

## Общий вывод

Нами была проведена работа по выбору региона для разработки месторождений. Мы:
* Загрузили, прочитали и изучили датасеты по каждому региону  
* Подготовили данные для построения моделей. Определили целевой признак, провели масштабирование остальных признаков  
* Разбили данные на тренировочную и валидационную выборки для каждого региона. Проверили себя, оценив их размеры
* Создали модели линейной регрессии для каждого региона. Сделали предсказания. Посчитали основные метрики.
* Отметили, что модель второго региона показала поразительное качество (метрика R2 составила 0,999). А наибольший средний предсказанный запас показал регион № 3.

In [19]:
display(average_reserves)

,Средний предсказанный запас
Регион,
1,92.592568
2,68.728547
3,94.965046


* Создали необходимы константы для дальнейшего расчета прибыли и рисков
* Рассчитали минимальное количество продукта в месторождениях, достаточное для безубыточной разработки скважины. Оно составило  
111 тыс. баррелей. Ближе всего по предсказанным значениям к этому показателю оказался третий регион.
* Подготовили функцию расчета прибыли, которую использовали для расчета прибыли методом bootstrap.
* Посчитали прибыль, 95 % доверительный интервал и вероятность появления убыточных точек в каждом регионе.  
  
**По результатам расчетов мы делаем вывод, что для разработки месторождений нужно предложить регион № 2, как регион с самой большой средней прибылью и с самым низким риском убытков.**